In [28]:
import pandas as pd
import seaborn as sns
from surprise import (Reader, Dataset, KNNWithMeans)

In [13]:
orginal_df = pd.read_csv("/mnt/data/public/bgg/bgg-19m-reviews.csv")

In [14]:
orginal_df = orginal_df.iloc[:,1:]

# EDA

In [15]:
orginal_df.describe()

,rating,ID
count,1.896481e+07,1.896481e+07
mean,7.081849e+00,1.101461e+05
std,1.595931e+00,9.309589e+04
min,1.401300e-45,1.000000e+00
25%,6.000000e+00,1.598700e+04
50%,7.000000e+00,1.075290e+05
75%,8.000000e+00,1.813040e+05
max,1.000000e+01,3.509920e+05


In [16]:
orginal_df["rating"].idxmin()

2615991

In [17]:
orginal_df.iloc[2615991,:]

user                                               GeoffMack
rating                                                   0.0
comment    Avoid this game.  Unless you like unusual game...
ID                                                        42
name                                      Tigris & Euphrates
Name: 2615991, dtype: object

In [18]:
duplicates = orginal_df[orginal_df.duplicated(subset=["user", "name"], keep=False)]

In [19]:
duplicates.sort_values(by="user")

,user,rating,comment,ID,name
3386467,-=Yod@=-,6.0,NaN,478,Citadels
9914865,-=Yod@=-,7.0,NaN,3201,Lord of the Rings: The Confrontation
7796340,-=Yod@=-,8.0,NaN,205398,Citadels
10456011,-=Yod@=-,7.5,NaN,18833,Lord of the Rings: The Confrontation
7731530,-Johnny-,8.0,Original. It has more than its fair share of p...,98,Axis & Allies
...,...,...,...,...,...
18662068,zzini,6.8,NaN,16719,Manhattan
10378681,zzyzewitsch,8.0,NaN,121,Dune
11068435,zzyzewitsch,7.0,NaN,283355,Dune
1916530,zzzabiss,8.0,Va muy bien como un filler introductorio. He r...,129622,Love Letter


In [20]:
sample = pd.read_csv("/mnt/data/public/bgg/games_detailed_info.csv")

/tmp/ipykernel_886/317015783.py:1: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  sample = pd.read_csv("/mnt/data/public/bgg/games_detailed_info.csv")


In [21]:
print(sample[sample["id"] == 129622])
print(sample[sample["id"] == 277085])

    Unnamed: 0       type      id  \
17          17  boardgame  129622   

                                            thumbnail  \
17  https://cf.geekdo-images.com/T1ltXwapFUtghS9A7...   

                                                image      primary  \
17  https://cf.geekdo-images.com/T1ltXwapFUtghS9A7...  Love Letter   

                                            alternate  \
17  ['Letters to Santa', 'List Miłosny', 'Lista Sk...   

                                          description  yearpublished  \
17  All of the eligible young men (and many of the...           2012   

    minplayers  ...  War Game Rank Customizable Rank Children's Game Rank  \
17           2  ...            NaN               NaN                  NaN   

   RPG Item Rank  Accessory Rank  Video Game Rank  Amiga Rank  \
17           NaN             NaN              NaN         NaN   

    Commodore 64 Rank Arcade Rank Atari ST Rank  
17                NaN         NaN           NaN  

[1 rows x 56 columns]


they just save the common names under names, but same name have different editions (example, the two rows below are the regular vs premium edition)

based on my opinion, i suggest to sort by item ID since premium editions may come at different price points/qualities

In [22]:
duplicates = orginal_df[orginal_df.duplicated(subset=["user", "ID"], keep=False)]

In [23]:
duplicates.sort_values(by="user")

,user,rating,comment,ID,name


if we sort by ID though, no duplicates

# Use Surprise (for neighborhood based)

In [30]:
orginal_df = orginal_df[["user","rating","ID"]]
orginal_df = orginal_df.rename(columns={"ID": "item"})

KeyError: "['ID'] not in index"

In [33]:
# Keep only games with at least 500 reviews
min_item_reviews = 500 
item_mask = orginal_df['item'].value_counts() >= min_item_reviews
item_over_min = orginal_df['item'].value_counts()[item_mask].index

# Optionally, filter users with at least 20 ratings
min_user_reviews = 20
user_mask = orginal_df['user'].value_counts() >= min_user_reviews
user_over_min = orginal_df['user'].value_counts()[user_mask].index

df = orginal_df[
        orginal_df['user'].isin(user_over_min) & 
        orginal_df['item'].isin(item_over_min)
]


In [32]:
reader = Reader(rating_scale=(0,10))
dataset = Dataset.load_from_df(df, reader)